# Computer Vision Project: The computer vision system detecting events in board game 'Man don't get angry'
### Authors
- Jędrzej Miczke
- Mateusz Nowicki

## Abstract  
The goal of the project is to introduce a computer vision system that tracks pawns and dice and detects events in the board game *'Man Don't Get Angry'*.  

### Game Overview  
The game consists of the following key elements:  
- A grid.  
- Four sets of four pawns, each set in a different color, representing each player.  
- A dice used to determine movements.  

The main goal for each player is to lead all their pawns to the "house." Key rules include:  
1. A pawn is introduced to the game only when the player rolls a 6.  
2. Players move their pawn of choice by the number rolled on the dice.  
3. If a player steps on an already occupied tile in the grid, the pawn that was  occuping  this tile previously is eliminated and must return to its base.  

### System Functionalities  
The functionalities of the system include:  
1. **Tracking the pawns.**  
2. **Tracking the dice.**  
3. **Reading the result of the dice roll.**  
4. **Controlling and updating the state of the game:**  
   - Monitoring pawns' positions.  
5. **Detection of game events:**  
   - Pawn entering the grid.  
   - Dice roll.  
   - Pawn movement.  

### Performance  
The system is designed with computational efficiency in mind, enabling it to operate in  real-time so that user can see
the output of the system while it processes


## Dataset
The dataset consists of  9 videos with the different levels of difficulty. In fact we can discern 3 levels of difficulty which I will briefly introduce and describe the differences between them.
1. **easiest level**:
- the grid is well alligned with the frame
- the lightning is stable throught the duration of the video and is roughly equal on the whole frame
- there are virtually no occlusions of the pawns  and the movement of the pawns are very slow
- the dice is rolled slowly 
2. **medium level**:
- the difference with regards to the easiest level is that the grid is not alligned with the frame, it can be arbitrarly 
rotated
- there appear shadows here and there
- there appear pawn occlusions

3. **hard level**:
- the shadows and light reflections are very strong
- the pawns are purposefully ocludded during their movement
- the  dice is rolled more frivolously, i.e throught the whole frame, and is rolled very fast
- additionaly, at this level we have introduced in some of the videos, a random objects laying on the grid

<br>
<br>
<br>

**Remark**:
 There is one more difficulty assosciated with our set-up. Mainly, the dice is being rolled not only on the grid but also on the background next to it  often. Therefore We couldn't just allign our grid with the reference pick( ie the grid alligning with the camera ) and detect everything on the alligned frame,as we would lost majority of the dice rolls. So we had to work in the perspective that we were given on the video.
<br>

## Below are the examples of frames in the described setting

### Frames from the beggining of the game for each level of difficulty 
- easiest level

![The easiest level](../images/frame_from_video_1.jpg)

- medium level 

![The medium level](../images/frame_from_video_2.jpg)

- difficult level 

![The hard level](../images/frame_from_video_3.jpg)

- difficult level , made even more difficult 

![The  hard level harder ](../images/frame_from_video_4.jpg)

![The  hard level harder ](../images/frame_from_video_5.jpg)


# frames from random moments  of  recordings for the different levels of difficulty


## the easy level

![The measy level](../images/frame_from_video_12.jpg)

As seen from the frame above , the easy videos are well allgned , without light changes  and without occlusions



## the medium level

![The medium level](../images/frame_from_video_9.jpg)
![The medium level](../images/frame_from_video_11.jpg)

As mentioned already earlier in the description of the data, here we can see the discriminnt features of the videos
with medium level of difficulty. So the grid is rotated, it does not allign with the camera. What's more there can occur
shaodows and quite strong light reflections, visible on the first picture of the medium frame above.


## the difficult level
![The hard level](../images/frame_from_video_6.jpg)

![The hard level](../images/frame_from_video_7.jpg)

![The hard level](../images/frame_from_video_8.jpg)

Here we are in the last category, the difficult examples. The light reflexes and shadows are changing and very strong. due to camera perspective the hand holding the pawns while moving may occlude a major part of the gird. Additionaly there are another obstacles as random objects put on the grid.







# The Workflow of the creation of the computer  vision system 

## Videos rescaling 
the script responsible for the rescaling of the video is held in the **video_rescaling.ipynb**
The basic idea was to rescale video to the smaller resolution i.e 512 X 512 for the reasons of the computational complexity, since the number of operations significantly decreases, and the loss of the information is not that significant for our purpose.


## Data annotation

The script used for the annotations is held in the **board_annotation.ipynb** The first step to make the work of the whole algorithm possible was creating the annotatations  of the grid for the game. We have diligently annotated every tile on the reference pick, so that the coordinates of the each tile are known. The annotations are stored in annotated_image.pkl and the corrections of some human mistakes commited while annotating are in anno.pkl
## Below is image how the annotated reference pick looks like

![The annotated im ](../images/annotated_im.png)

## Here is an excerpt of the anotations: 
basically a hashmap tile_name : coordinates_of_vertices

In [12]:
import pickle
with open('../resources/annotated_image.pkl', 'rb') as f:
    loaded_annotations = pickle.load(f)

for key, value in list(loaded_annotations.items())[:10]:
    print(f"{key}: {value}")


blue_start: [[[280, 594], [280, 554], [323, 557], [320, 594]]]
grid_1: [[[281, 553], [281, 516], [317, 514], [321, 555]]]
grid_2: [[[279, 516], [280, 476], [320, 476], [319, 516]]]
grid_3: [[[281, 476], [281, 438], [319, 436], [319, 476]]]
grid_4: [[[280, 476], [241, 474], [241, 437], [280, 437]]]
grid_5: [[[240, 475], [201, 475], [203, 436], [241, 436]]]
grid_6: [[[201, 436], [203, 398], [243, 399], [243, 434]]]
grid_7: [[[203, 399], [203, 358], [243, 359], [243, 400]]]
grid_8: [[[201, 398], [164, 398], [164, 360], [203, 358]]]
grid_9: [[[162, 397], [125, 398], [125, 360], [165, 360]]]


## Aligning the Frame
As mentioned, we have annotated the entire frame. Since all the tiles on the grid of the reference frame are annotated, we only need to determine the translation, rotation, and scaling (zoom) between the reference frame and each video. Once this relationship is established, we can obtain the coordinates for all grid tiles in all videos.

To find these transformations, we used the SIFT algorithm. We identified all the keypoints between the frames and selected the 200 best matches. The number 200 was chosen arbitrarily as it seemed large enough to ensure accuracy while accounting for possible occlusions of keypoints on the grid.

For these 200 matches, we had their coordinates on both the annotated reference frame and the video frame. Using this data, we computed the homography matrix with the following function:
cv2.findHomography(pts_frame, pts_ref, cv2.RANSAC, 5.0).

With the homography matrix, we were able to transform the annotations from the reference frame to the video frame.
## Below the image of of keypoint finding 

### The keypoints matching between the frames
![keypoints ](../images/keypoints_found.jpg)

### The calculated transformed annotations overlayed on some frame
![annotatiosn transformed ](../images/annotations_transformed.png)

AS seen our appraoch works verly well even in quite an acute angle and with additional distracting object(wallpaper knife)

## Colorfinder

This is a backbone of our computer vision  system. The class Colorfinder is reponsible for filtering the video in the given color range and setting to 0 all the values outside this color range. It works in the 3 colorspace BGR, HSV, HSL and two modes: debug and production. In the debug mode, there is a trackbar available which enables the user to change the color ranges for the given video in the real time. The second mode, uses values passed to the function by the user and filters every frame as mentioned earlier. The natural usage of this tool is that firstly for each object: red pawn, blue pawn, green pawn, dice we run the Colorfinder in the debugg mode to determine the values of this object , and when determined it is run with these values set on the rest of the videos. Detecting red pawns, Detecting blue pawns, detecting yellow pawns, detecting green pawns, detecting dice, detecting blue pawns in base and detecting red pawns in base are all done basing on the colorfinder color filtering.
### **For colorfinder demo, please look at colorfinder_Demo video**

## Pawns filtering

There are 6 different functions  related to pawns based on ColorFinder which are responsible respectively for:
- detecting red pawns in the base and on the start( this done separately from detecting the red pawns on other grid tiles  since it was not possible to find  combination of parameters which would find red pawns both on the yellow grid and in red base witout returning many false positive)
- detecting red pawns everywehre else on the grid
- detecting blue pawns in the base and on the start( reason for this the same as the one for red base detection mentioned above )
- detecting blue pawns everywehre else on the grid 
- detecting yellow pawns on the whole grid 
- detecting green pawns on whole grid

As mentioned above the pawns filtering is based on the colorfinder funcionalities, the proper values in HSV or HLS or GBR are filtered to retain only the desired objects. However due to differences of lightning, angles of camera, shadows, this tool itself wont work perfectly for each video. It could be easily overfitted to some of the videos to filter everything perfectly, however the idea is to strive for more of an universal application of the color vision system. To achieve this goal, after the color filtering other necessary operations take place, Mainly:
- **Area of interest filtering ** , as we know the rules of the Man don't get angry, we now that the pawns are supposed to move only on the predefined tiles, and the board consists also of some background elements, embellishments-dragon graphic in the middel, etc, all redundant for our detections. So to surpress the risk of false positive detection we have restricted the area of interest of the detections to valid tiles on the board.
- **thresholding**: the image is turned to grayscale and then thresholded , using the **OTSU** thresholdoing as empirically it proved to perform the best , better than adaptive or fixed value thresholding.
- **Morphological operations** Morphological operations were necessary, **opening** for deletion of the noise, **closing**for the closing the remaining contours, again **opening** to delete the remaining noise, and at last in some cases also **dilation** to emphasize the remaining structure(the filtered pawns)
- **Contour finding and filtering**
On such preprocessed image , now a binary one, we perfrom the contour detection with the assumptions that the found contours will represent our desired object that we search for. However this is not true without defining further conditions, becausee we still need to get rid of some noise. Therefore we filter contours on the given criteria:
-area: the area must be above some threshold,
-circularity: the calculated circularity of contour also must be above some set threshold since pawns are generally quiete circular structures especially from the birds-eye perspective
- number: for the case of yellow and green when we search for the pawns at whole grid at  once, we can restrict the desired number of contours to 4, as we know we have only 4 pawns. And we choose 4 biggest contours from the found ones.

### **For presentation example presentation of the color filtering please look at yellow_pawns_filtered video**

## Detection and tracking of the dice

the detection of the dice also bases on the ColorFinder filtering of the video, however the approach needed to be  refined in order to work decently.  Apart from the filtering the video based on color, the following operations take place:
- **Background subtraction**, the idea was that we search for the dice only on the background subtracted frame(dice moves when rolled) and upon finding it we set the **KCF tracker** on the original frame in the coordinates of found contour in order  to track it. Such steps must be taken, since otherwise we would be plagued with the false positive detections that stem for example from the light reflections and background elements of  basically same color as dice . 
- Additionaly due to the difficulties we imposed on ourselves in the difficult examples(objects thrown on the board), we had to take one more smart step. Mainly, before  the processing of the video we take the first frame, blurr it and mask the regions which fall in the desired color ranges for dice as excluded from the search, thus surpressing the false positive rate substantially
- Lastly we have set a tracker reinitialization every n=9 frames , as it is possible that anyway the tracker can get lost , and become tracking some undesired patch of the board (light reflection for example) whereas the dice is rolling completly elswehere. for this case we set the tracking to false, and again search for the appropriate dice contours on the color filtered background subtracted video.
- Finally since we know there is only one dice, we also take only the biggest found contour 

### **For the presentation of background filtering for the dice please look at the dice_tracking video**

## Pips detection
The pips are detected on the dice when it remains stable. It means when the tracker has not lost it for n = 5 frames and it has not moved, meaning the absolute difference of the bounding boxes coordinates of the tracked object in the subsequent frames are below some small threshold. The techinque used to extract the pips is not very elaborate. Basically an inverse binarization with the otsu thresholding takes place(since pips are black and dice white) in the bounding box of the original frame indicated by the tracker, and the contours which fall in some predefined area and circularity range are deemed a valid pips and counted.

# State of the game control 

This part is is the most elaborate part of the system , because it's goal is to infer from somewhat erratic detections the state of the game with resonable stability.  many auxillary hashmaps are used to faciliate the process, mainly: states hashmap, states history hashmap, ensured states hashmap, ensured history hashmap for each color. 

The states hashmap: the current state of the tiles on the grid -occupied or not , and by which pawn color

The states history hashmap: stores the same information what states hashmap but for the last 120 frames.

the ensured states hashmap: stores the same as the the state hashmap but after heursitically ensuring that there are 4 pawns in the detections for each color. 

The ensured  states history:  the history of ensured states


- The detections of the pawns positions are done for each frame. The main detection logic is as following: We look at the bounding boxes returned from the functions responsible for finding  given type of pawns and find for each such bounding box coordinates of the tile for which the overlapp with this bounding box is the biggest. We assume the pawn labeld with this bounding box must be on the tile with the biggest overlap

But also an additional step to ensure the correctenss must be  taken. Mainly the detection for each color are forced to be 4 , because as we know we  must have  as much as 4 pawns. So in case there are less pawns  detected for the given color, then we look in the history of the grid states(which is stored for n =120 last frames), and see which grid was occupied most recently which is not detected  as occupied now and we infer a pawn must be there. On the other hand, in case there are more than 4 detected pawns of given color, we must decide to forfeit some of the detections. In this case, firstly we look into the base for given color, an action  dictated by pragmatism, the  biggest amount of the false positive are commited there(very similar  color of tile to the pawn) and see which tile which is detected now as occupied was occupied the last time the longest time ago.


# Event Detections
-Dice roll:
starting from the simplest event, the dice roll. As I have described earlier we read the outcome of the dice roll only after it achieves the stability.Upon the reading , we assume the dice is in the stable state, as long as the bounding boxes of tracked dice in the subsequent frames dont differ by some threshold value. Until then we assume it was all a part of a one dice roll. So basically we track the dice,until it stops in one place, And when it moves again we assume it is already a part of the next roll

-Pawn movement :
The detection of this event is far more problematic in our case. As I mentioned before, due to the nature of the grid: pawns on the tiles of the same color, yellow pawns always on the yellow , the detections of the pawns are not utterly stable, they tend to be erratic, flicker, the detection's bounding box can suddenly vanish in the shadow, the pawn can be utterly occluded by the hand during the alligning or moving of the pawns, or a false positive detection can suddenly pop up in the light reflection. Therefore deciding whether a pawn has really moved or not was a difficult task in such a turbulent environment. In our tries to address this task we created a system consisting of several rules. 
1. a movement can only happen if the tile of the board which in the given frame is occupied was not occupied for n=30 previous frames. This rule ensures that we will not detect many false positive events due to the erratic nature of the detections. Because if an object is present but , is occluded for a second , and then appears again in the camera, we do not want to detect it  as the second movement of the same pawn again.
2. A movement can only happen if in recent history consisting of k=80 frames in at least p=60  frames the given frame was  occupied. Again a condition against the instability of detections. tightening the requirements from the first condition.
3.   Lastly, movement of pawn can not happen to the same square the last movement of this pawn was made to. Again a condition against flickering of the detections

- Pawn entering the game
Basically the same logic as detection of the pawns movements ,only difference that it  triggers when pawn's movement to the tile 'start_GIVEN_COLOR' is  detected .



# Outcomes and conclusions
### all the outcomes are available on the google disc and appropriately labeled


In general we are satisfied with the system performance, especially regarding its time effectiveness as it is nearly  capable of processing the videos in the real time. The performance is resonably universal among the videos, not manifesting  any strong overfitting tendencies towards any particular videos. Moreover the state detection seemed to add the quality of the raw detections as due to the various stability thresholds it is less partial to temporary false readings , occlusions or rapid light changes.  Surely the level of the difficulty influences the quality of the detection system, the strong changing shadowing seemed to pose the biggest problem, making the system event detections very erratic at times at extreme examples such as hard_3 video, however the position tracking of pawns and dice still seemed to retain resonable accuracy. 

However, surely there are some flaws in the system that could be adressed to improve it's performance
- 1 the dice detection has problem with the detection of 6 , often undercounting and counting it as  2 or 4 . It is probably  caused by the fact that in the 6 the pips are the most densly packed, and in such a small resolution that we process the video in, there is a possibility that these pips blend together in 2 or 3 contours.
- 2 The yellow detections suffer substantially in the strong shadowing which is visible at hard_3 video, however it is really difficult to prevent such situation sticking to the techiques that we have used, since the grid is generally in the similar color as yellow pawns and especially when grid  falls in the shadows it gains nearly the same colorimetric parameters as yellow pawns in normal lightning have ,causing many false positive readings. One possible solution would be to detect the desired elements only after ensuring that the hand is not visible on the frame, so firstly hand detection, and then in case it is missing we could follow with the predictions of  other elements.
- 3 Another issue, despite the preventive measures that we have undertaken , there still happen quite often false positive reading of dice. In case that the dice is not present at the screen but there are strong light reflections visible then the system tends to mistake them for the dice. 
- 4 Lastly, the finding of the pawns in the houses, so the places where all the players wish to move their pawns to is very hard due to nearly same color of the desired tiles as the pawns, so we were not able to do it with resonable accuracy.
